In [1]:
import sys
print(sys.argv[0])
import math
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import numpy as np


np.random.seed(123)
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from keras.layers import LSTM, GRU, Bidirectional
from keras.models import Sequential
from keras.utils import np_utils
from keras.models import Sequential
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import functools


/home/danny/.local/lib/python3.8/site-packages/ipykernel_launcher.py


Using TensorFlow backend.


In [2]:
def load_dict_from_hdf5(filename):
    with h5py.File(filename, 'r') as h5file:
        return recursively_load_dict_contents_from_group(h5file, '/')
    h5file.close()

def recursively_load_dict_contents_from_group(h5file, path):
    ans = {}
    for key, item in h5file[path].items():
        if isinstance(item, h5py._hl.dataset.Dataset):
            ans[key] = item.value
        elif isinstance(item, h5py._hl.group.Group):
            ans[key] = recursively_load_dict_contents_from_group(h5file, path + key + '/')
    return ans
def convert_tensor(arg):
    return tf.convert_to_tensor(arg, dtype=tf.float32)



In [3]:
#from google.colab import drive
#drive.mount('/content/drive/')
#filename = "/content/drive/My Drive/Colab/2011+2012.hdf5"
#filename2 = "/content/drive/My Drive/Colab/2012_valid.hdf5"
#data = load_dict_from_hdf5(filename)
#vdata = load_dict_from_hdf5(filename2)

In [4]:
filename = "/home/danny/Projects/crohme-data-extractor/train.hdf5"
train_data = load_dict_from_hdf5(filename)

filename = "/home/danny/Projects/crohme-data-extractor/validation.hdf5"
val_data = load_dict_from_hdf5(filename)

<ipython-input-2-74920beda866>:10: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  ans[key] = item.value


In [5]:
label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False)
unique_labels = {}

def to_lists(data):
    labels_lst = []
    curves_lst = []
    max_curves = 0
    for a in data:
        for b in data[a]:
            label = data[a][b]['label']
            if(label not in unique_labels): unique_labels[label] = 1
            labels_lst.append(label)
            sample = np.concatenate(list(data[a][b]['feat_bez_curves'].values()))
            sample = np.nan_to_num(sample)
            curves_lst.append(sample)
            max_curves = max(max_curves,sample.shape[0])
    return curves_lst, labels_lst, max_curves
        
#From here: https://stackoverflow.com/questions/57346556/creating-a-ragged-tensor-from-a-list-of-tensors
def stack_ragged(tensors):
    values = tf.concat(tensors, axis=0)
    lens = tf.stack([tf.shape(t, out_type=tf.int64)[0] for t in tensors])
    return tf.RaggedTensor.from_row_lengths(values, lens)

def stack_dense(tensors,max_curves):
    pad = lambda x:np.pad(x, pad_width=((0,max_curves-len(x)),(0,0)))
    return np.concatenate([np.expand_dims(pad(x),0) for x in tensors],axis=0)
def encode_labels(labels,fit=True):
    if(fit): label_encoder.fit(labels)
    integer_encoded = label_encoder.transform(labels)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    if(fit): onehot_encoder.fit(integer_encoded)
    onehot_encoded = onehot_encoder.transform(integer_encoded)
    return onehot_encoded


In [10]:
#Process train_data
curves_lst, labels_lst, max_curves = to_lists(train_data)
#X_train = stack_ragged(curves_lst)
X_train = stack_dense(curves_lst,max_curves)
Y_train = encode_labels(np.array(labels_lst))

#This is just a for a sanity check
x_unq = unique_labels
A = label_encoder.transform(sorted(list(x_unq.keys())))
unique_labels = {}

#Process val_data
curves_lst, labels_lst, max_curves = to_lists(val_data)
#X_val = stack_ragged(curves_lst)
X_val = stack_dense(curves_lst,max_curves)
Y_val = encode_labels(np.array(labels_lst),fit=False)

#This is just a for a sanity check
B = label_encoder.transform(sorted(list(unique_labels.keys())))



In [7]:
print(A)
print(B)
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100]
(102772, None, 9)
(102772, 101)
(12487, None, 9)
(12487, 101)


In [8]:
#Two samples labelled '0'
print(X_train[102737]) 
print(X_val[43])

tf.Tensor(
[[ 14.08450704  18.30985915  -9.85915493  81.69014085   0.12010874
   -0.9055069   21.78027361  30.09209851   1.        ]
 [  4.22535211 100.          25.35211268 -14.08450704  -0.5070985
   -0.27829966  12.0068625   10.54683918   0.        ]
 [ 29.57746479  85.91549296  11.26760563 -85.91549296  -0.19134725
   -0.13040331  32.67898951  26.98834631   0.        ]
 [ 40.84507042   0.         -22.53521127  14.08450704  -0.55859932
    0.55859932  14.21732925   6.27275509  -1.        ]], shape=(4, 9), dtype=float64)
tf.Tensor(
[[ 1.86046512e+01  9.30232558e+00 -1.86046512e+01  1.62790698e+01
   0.00000000e+00  1.66533454e-16  8.24042311e+00  8.24042311e+00
   1.00000000e+00]
 [ 0.00000000e+00  2.55813953e+01  3.95348837e+01  6.51162791e+01
  -5.45655055e-01 -7.03390718e-01  3.41140436e+01  3.58710836e+01
   0.00000000e+00]
 [ 3.95348837e+01  9.06976744e+01  6.04651163e+01 -4.65116279e+01
  -6.55695626e-01 -9.15100701e-01  3.29741638e+01  4.11668451e+01
   0.00000000e+00]
 [ 1.00

In [11]:
#Best so far Bi-GRU-100-drop20-recdrop00: 87.7%
seq = [
    tf.keras.Input(shape=(None,9),ragged=False),
    tf.keras.layers.Masking(mask_value=0),
    tf.keras.layers.GRU(120,dropout=0.2, recurrent_dropout=0.1, reset_after=False),
    tf.keras.layers.Dense(101, activation = "softmax"),
]
seq_no_masking = seq[:1] + seq[2:]

keras_model = tf.keras.Sequential(seq)

top5_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=5)

top5_acc.__name__ = 'top5_acc'

keras_model.compile(loss = 'categorical_crossentropy',  optimizer='adam', metrics = ['accuracy', top5_acc])

keras_model.fit(X_train, Y_train, epochs = 25 , batch_size = 32, validation_data=(X_val, Y_val))
print(keras_model.summary())


Epoch 1/25
3212/3212 [==============================] - 126s 39ms/step - loss: 1.4546 - accuracy: 0.6481 - top5_acc: 0.8558 - val_loss: 0.8949 - val_accuracy: 0.7785 - val_top5_acc: 0.9276
Epoch 2/25
3212/3212 [==============================] - 126s 39ms/step - loss: 0.7530 - accuracy: 0.8011 - top5_acc: 0.9522 - val_loss: 0.7139 - val_accuracy: 0.8109 - val_top5_acc: 0.9481
Epoch 3/25
3212/3212 [==============================] - 127s 40ms/step - loss: 0.6504 - accuracy: 0.8210 - top5_acc: 0.9634 - val_loss: 0.6811 - val_accuracy: 0.8210 - val_top5_acc: 0.9530
Epoch 4/25
3212/3212 [==============================] - 131s 41ms/step - loss: 0.5850 - accuracy: 0.8368 - top5_acc: 0.9689 - val_loss: 0.6160 - val_accuracy: 0.8378 - val_top5_acc: 0.9632
Epoch 5/25
3212/3212 [==============================] - 135s 42ms/step - loss: 0.5515 - accuracy: 0.8441 - top5_acc: 0.9725 - val_loss: 0.6010 - val_accuracy: 0.8326 - val_top5_acc: 0.9641
Epoch 6/25
3212/3212 [==============================] -

In [12]:
predict_model = tf.keras.Sequential(seq_no_masking)
predict_model.set_weights(keras_model.get_weights()) 
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(predict_model, 'danny_modeljs')

/home/danny/.local/lib/python3.8/site-packages/tensorflowjs/converters/keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [ ]:
keras_model.save("danny_model")

In [ ]:
keras_model.save_weights("model.h5")

In [ ]:
import timeit

N=1000
def time_ms(f):
    f() #warm start
    return " %0.6f ms" % (1000.0*(timeit.timeit(f, number=N)/float(N)))
one_character = X_val[0:1]

In [ ]:
def pred():
    keras_model.predict(one_character)
    
print(time_ms(pred))

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import GRU, Attention, Input, Concatenate, Dense, TimeDistributed, Activation, dot

In [36]:




max_curves = 60
max_symbols = 40
curve_size = 9
alphabet_size = 101
hidden_size = 100

garbage_X = np.random.random((1000,max_curves,curve_size))
garbage_Y = np.random.random((1000,max_symbols,alphabet_size))


###############################Training Model#######################################
encoder_inputs = Input(batch_shape=(None, max_curves, curve_size), name='encoder_inputs')
decoder_inputs = Input(batch_shape=(None, max_symbols, alphabet_size), name='decoder_inputs')


encoder_gru = GRU(hidden_size, return_sequences=True, return_state=True, reset_after=False, name='encoder_gru')
encoder_out, encoder_state = encoder_gru(encoder_inputs)
print("E",encoder_out.shape)
print("Es",encoder_state.shape)

decoder_gru = GRU(hidden_size, return_sequences=True, return_state=True, reset_after=False, name='decoder_gru')
decoder_out, decoder_state = decoder_gru(decoder_inputs, initial_state=encoder_state)
print("D",decoder_out.shape)

#attn_layer = Attention(name='attention_layer')
#attn_out = attn_layer([encoder_out, decoder_out])

attn_out = dot([decoder_out,encoder_out], axes=[2, 2])
attn_out = Activation('softmax')(attn_out)

print("A", attn_out.shape)
context = dot([attn_out, encoder_out], axes=[2,1])
print("C",context.shape)

decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_out, context])

dense = Dense(alphabet_size, activation='softmax', name='softmax_layer')
dense_time = TimeDistributed(dense, name='time_distributed_layer')
decoder_pred = dense_time(decoder_concat_input)
print("P",context.shape)


training_model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_pred)
training_model.compile(optimizer='adam', loss='categorical_crossentropy')



###############################Encoder Inference#######################################
inf_encoder_model = Model(inputs=[encoder_inputs], outputs=[encoder_out, encoder_state])
inf_encoder_model.compile()

#######################Decoder Inference w/ attention###############################
inf_decoder_state = Input(batch_shape=(None, hidden_size), name='inf_decoder_state')
inf_decoder_input = Input(batch_shape=(None, 1, alphabet_size), name='inf_decoder_input')
inf_encoder_inout = Input(batch_shape=(None, max_curves, hidden_size), name='inf_encoder_inout')

print("Di",inf_decoder_input.shape)
print("Ds",inf_decoder_state.shape)

_inf_decoder_out, _inf_decoder_state = decoder_gru(inf_decoder_input, initial_state=inf_decoder_state)
print("iD",_inf_decoder_out.shape)
print("Eio",inf_encoder_inout.shape)
inf_attn_out = dot([_inf_decoder_out,inf_encoder_inout], axes=[2, 2])
inf_attn_out = Activation('softmax')(inf_attn_out)

print("iA",inf_attn_out.shape)

inf_context = dot([inf_attn_out, inf_encoder_inout], axes=[2,1])
print("iC", inf_context.shape)

inf_decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([_inf_decoder_out, inf_context])

#dense = Dense(alphabet_size, activation='softmax', name='softmax_layer')
#dense_time = TimeDistributed(dense, name='time_distributed_layer')
inf_decoder_pred = dense_time(inf_decoder_concat_input)

inf_decoder_model = Model(inputs=[inf_encoder_inout,inf_decoder_input,inf_decoder_state], outputs=[inf_decoder_pred,_inf_decoder_state,inf_attn_out])
inf_decoder_model.compile()
#inf_decoder_model = Model(inputs=[encoder_out, encoder_state], outputs=[?])
#inf_decoder_input = Input(batch_shape=(None, hidden), name='decoder_inputs')


E (None, 60, 100)
Es (None, 100)
D (None, 40, 100)
A (None, 40, 60)
C (None, 40, 100)
P (None, 40, 100)
Di (None, 1, 101)
Ds (None, 100)
iD (None, 1, 100)
Eio (None, 60, 100)
iA (None, 1, 60)
iC (None, 1, 100)


In [4]:
start_vec = np.zeros(101,dtype=np.float32)
np.put(start_vec,100,1)
start_vec = tf.constant(start_vec.reshape(1,1,alphabet_size))
end_vec = tf.constant(np.array([0,0,0,0,0,0,0,0,1],dtype=np.float32))
print(start_vec.shape)

(1, 1, 101)


In [37]:
#DOESN"T WORK YET
def predict(curves):
    '''
    encoder_out, encoder_state = encoder_gru(tf.constant(curves), initial_state=tf.zeros([1,hidden_size]))
    '''
    encoder_out, encoder_state = inf_encoder_model.predict(tf.constant(curves))
    print("E",encoder_out.shape)
    decoder_inp, decoder_state = start_vec, encoder_state
    
    
    out_symbols = []    
    for i in range(10):
    #while True:
        '''
        print("D-inp",decoder_inp.shape)
        print("D-state",decoder_state.shape)
        decoder_out, decoder_state = decoder_gru(decoder_inp, initial_state=decoder_state)
        print("D",decoder_out.shape)
        attn_out = dot([decoder_out,encoder_out], axes=[2, 2])
        attn_out = Activation('softmax')(attn_out)
        
        print("A",attn_out.shape)
        
        context = dot([attn_out, encoder_out], axes=[2,1])
        print("C", context.shape)

        decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_out, context])

        dense = Dense(alphabet_size, activation='softmax', name='softmax_layer')
        dense_time = TimeDistributed(dense, name='time_distributed_layer')
        decoder_pred = dense_time(decoder_concat_input)
        print("P", decoder_pred.shape)
        '''
        decoder_pred, decoder_state, attention = inf_decoder_model.predict([encoder_out,decoder_inp, decoder_state])
        
        max_index = tf.argmax(decoder_pred, -1)
        decoder_inp = tf.one_hot(max_index,alphabet_size)
        out_symbol = max_index
        print(attention)
        #decoder_inp = tf.expand_dims(tf.argmax(decoder_pred, -1), 0)
        print(out_symbol)
        out_symbols.append(out_symbol)
        
        
       # encoder_out = 
        #out_words.append(fr_tokenizer.index_word[de_input.numpy()[0][0]])
        
        #alignments.append(alignment.numpy())

        #if out_words[-1] == '<end>' or len(out_words) >= 20:
        #    break

    print(' '.join([str(x.numpy()) for x in out_symbols]))
    return out_symbols

In [38]:
training_model.fit([garbage_X, garbage_Y], garbage_Y, epochs = 2, batch_size = 32)

Epoch 1/2
32/32 [==============================] - 2s 51ms/step - loss: 233.2841
Epoch 2/2
32/32 [==============================] - 2s 55ms/step - loss: 232.4706


In [39]:
predict(garbage_X[0:1])

E (1, 60, 100)
[[[0.01453928 0.01487747 0.015997   0.01637685 0.01530292 0.01555423
   0.01643004 0.0169257  0.01579296 0.01662014 0.01641673 0.01645177
   0.01735806 0.01699404 0.01803191 0.01697245 0.01678746 0.01675483
   0.01664654 0.016813   0.01772239 0.01721153 0.01621451 0.01621397
   0.01719792 0.0173689  0.01685258 0.0164619  0.01702885 0.01775679
   0.01727645 0.01696535 0.0168335  0.01694597 0.01725658 0.01653496
   0.01703662 0.01690287 0.016323   0.01626301 0.01715383 0.01718754
   0.0170712  0.016134   0.01668269 0.01623712 0.0161945  0.0165122
   0.01617979 0.0161477  0.01651124 0.01620462 0.01629668 0.01569291
   0.01713289 0.01736034 0.0169364  0.01712453 0.01800999 0.01721877]]]
tf.Tensor([[97]], shape=(1, 1), dtype=int64)
[[[0.01543377 0.0153101  0.01601563 0.01652016 0.0158517  0.01599865
   0.01650342 0.01697299 0.01641552 0.01669809 0.01643051 0.01668394
   0.0167626  0.01659419 0.01745388 0.01701357 0.01679508 0.01692035
   0.01647742 0.01677028 0.01722088 0.016

TypeError: sequence item 0: expected str instance, tensorflow.python.framework.ops.EagerTensor found

In [12]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(full_model, 'danny_nmt_modeljs')